# Fear &amp; Greed Index vs BTC Price Analysis

This notebook loads the processed dataset, fits a linear regression model (using the same logic as the training script), and visualizes the relationship between the fear &amp; greed index, BTC price, and the model's predicted price.

In [1]:
# Import libraries
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from pathlib import Path


ModuleNotFoundError: No module named 'polars'

In [ ]:
# Load the processed data
repo_root = Path(__file__).resolve().parents[1]
data_path = repo_root / ".." / "data" / "processed" / "fear_and_greed_history_5min.parquet"
df = pl.read_parquet(data_path).drop_nulls()
df = df.with_columns([
    pl.col("open_actual_value").cast(pl.Float64),
    pl.col("close_bitcoin_price_usd").cast(pl.Float64)
])
df = df.filter(~pl.col("open_actual_value").is_null() & ~pl.col("close_bitcoin_price_usd").is_null())
df = df.sort("timestamp")
df.head()


## Fit Linear Regression Model (reuse training logic)

We use the same features as in the training script to fit a model and generate predictions.

In [ ]:
# Prepare features and target
feature_cols = [
    "open_actual_value",
    "open_bitcoin_price_usd",
    "high_bitcoin_price_usd",
    "low_bitcoin_price_usd",
    "avg_actual_value",
]
X = df.select(feature_cols).to_numpy()
y = df["close_bitcoin_price_usd"].to_numpy()

# Fit model
model = LinearRegression()
model.fit(X, y)

# Predict
df = df.with_columns([
    pl.Series("predicted_price", model.predict(X))
])
df.head()


## Visualize: Fear &amp; Greed Index vs BTC Price vs Predicted Price

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df["timestamp"].to_list(), df["close_bitcoin_price_usd"].to_list(), label="BTC Close Price", color="blue")
plt.plot(df["timestamp"].to_list(), df["predicted_price"].to_list(), label="Predicted Price", color="orange", linestyle="--")
plt.plot(df["timestamp"].to_list(), df["open_actual_value"].to_list(), label="Fear & Greed Index (Open)", color="green", alpha=0.5)
plt.legend()
plt.title("BTC Price, Predicted Price, and Fear & Greed Index Over Time")
plt.xlabel("Timestamp")
plt.ylabel("Value")
plt.tight_layout()
plt.show()


## Correlation Analysis

Let's check the correlation between the fear &amp; greed index and BTC price.

In [ ]:
cor_fg_btc = np.corrcoef(df["open_actual_value"].to_numpy(), df["close_bitcoin_price_usd"].to_numpy())[0, 1]
cor_pred_btc = np.corrcoef(df["predicted_price"].to_numpy(), df["close_bitcoin_price_usd"].to_numpy())[0, 1]
print(f"Correlation (Fear & Greed, BTC Close): {cor_fg_btc:.3f}")
print(f"Correlation (Predicted, BTC Close): {cor_pred_btc:.3f}")
